加载文档

In [1]:
import os
import glob
import json
import logging
import fitz  # PyMuPDF, 用于PDF文本提取
from dotenv import load_dotenv  # 用于加载.env环境变量
from langchain.vectorstores import Chroma  # 向量数据库
from langchain.embeddings import SentenceTransformerEmbeddings  # 向量化
from langchain.text_splitter import CharacterTextSplitter  # 文本分片
from langchain.llms import OpenAI  # OpenAI LLM调用

# 加载本地.env文件中的环境变量（如API KEY）
load_dotenv()

# 读取OpenAI密钥
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
# 设置使用的模型（OpenRouter上的GPT-4o-mini）
OPENAI_MODEL = "gpt-4o-mini"

# 配置日志，方便调试
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

def load_pdf_from_folder(folder_path):
    """
    Load all the PDF files in the specified folder, 
    returning the full text content of each file (string list).
    """
    all_documents = []
    # Iterate over all pdf files
    for filename in glob.glob(f"Local_knowledge_base/*.pdf"):
        document_text = ""
        # Open the PDF and iterate over each page, accumulating the text
        with fitz.open(filename) as doc:
            for page_num in range(doc.page_count):
                page = doc.load_page(page_num)
                document_text += page.get_text("text")
        all_documents.append(document_text)
    return all_documents


# 调用函数并打印
all_documents = load_pdf_from_folder("Local_knowledge_base")  # 传入PDF所在文件夹路径
print("Loaded documents:", len(all_documents))
print("First document text:", all_documents[0][:100])  # 打印第一个文档的前100个字符
print("Second document text:", all_documents[1][:100])  # 打印第二个文档的前100个字符

Loaded documents: 4
First document text: 1
CSRF
1. 原理
2. 与XSS区别
3. 常见场景
4. 常见漏洞点
5. 漏洞危害
6. CSRF Poc 构造
7. 漏洞审计
8. 漏洞修复
9. Webgoat
跨站请求伪造（Cro
Second document text: 1
XXE 漏洞
1. xml基本介绍
1.1. 什么是xml
1.2. xml 内容示例
1.2.1. DTD 约束
1.2.2. 内部实体 Internal Entity
1.2.3. 外部实体 


分片

In [2]:
from langchain.schema import Document
from langchain.text_splitter import CharacterTextSplitter
from langchain.text_splitter import RecursiveCharacterTextSplitter

def split_documents(documents, chunk_size=1000, overlap=100):
    """
    对每个文档做分片，chunk_size控制单片长度，overlap保证上下文连贯
    输入: 字符串list，输出: 分片list
    """
    # 将字符串列表转换为Document对象列表
    doc_objects = [Document(page_content=doc) for doc in documents]
    
    # 创建text_splitter对象
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=overlap)
    
    # 分片并返回结果
    return text_splitter.split_documents(doc_objects)

# 使用示例
split_docs = split_documents(all_documents, chunk_size=1000, overlap=100)
print(split_docs)
# print("Number of document chunks:", len(split_docs))
# 使用示例
print("First chunk text:", split_docs[0].page_content[:100])  # 打印第一个分片的前100个字符
print("Second chunk text:", split_docs[1].page_content[:100])  # 打印第二个分片的前100个字符


[Document(metadata={}, page_content='1\nCSRF\n1. 原理\n2. 与XSS区别\n3. 常见场景\n4. 常见漏洞点\n5. 漏洞危害\n6. CSRF Poc 构造\n7. 漏洞审计\n8. 漏洞修复\n9. Webgoat\n跨站请求伪造（Cross-site request forgery）CSRF，是一种使已登录用户在不知情的情况下执行某种\n动作的攻击，利用用户已登录身份，伪造请求操作。\n实际上这种方式是攻击者通过一些钓鱼等手段欺骗用户去访问一个自己曾经认证过的网站，然后执行一\n些操作（如后台管理、发消息、添加关注甚至是转账等行为）。浏览器曾经认证过，因此被访问的网站\n会认为是真正的用户操作而去运行。CSRF 漏洞的工作原理是攻击者盗用了用户的身份， 以用户的名义\n发送恶意请求。因为攻击者看不到伪造请求的响应结果，所以CSRF攻击主要用来执行动作，而非窃取用\n户数据。\n当受害者是一个普通用户时，CSRF可以实现在其不知情的情况下转移用户资金、发送邮件等操作；但是\n如果受害者是一个具有管理员权限的用户时CSRF则可能威胁到整个Web系统的安全。\n1. 原理\n2'), Document(metadata={}, page_content='一次完整的 CSRF 攻击需要具备以下两个条件：\n用户已经登录某站点，并且在浏览器中存储了登录后的 Cookie 信息。\n在不注销某站点的情况下，去访问攻击者构造的站点。\n例：\n网站管理员添加用户的 url 链接是“add?name=admin1&password=admin1”，如果在添加用户处有 \nCSRF 漏洞，那么当攻击者把“add?name=hack&password=hack”这个链接发给网站管理员，而管理员\n点击了这个链接，那么就会在管理员并不知情的情况下给网站创建了”hack“账户。\n从信任的角度来区分\nXSS：利用用户对站点的信任\nCSRF：利用站点对已经身份认证的信任\nCSRF 攻击可能出现的场景有很多，如更改个人信息、修改密码、添加/修改资料、关注/取关用户、发\n布主题或信息、与交易相关的操作等。\n1. CSRF 基于 post 请求添加账号\n2. CSRF 修改密码 \n●\n●\n2. 与XSS区别\n3. 常

索引

In [3]:
from langchain.vectorstores import Chroma
from langchain.embeddings import SentenceTransformerEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.schema import Document

def build_index(documents):
    """
    Convert the sharded documents into vectors and index them
    """
    embeddings = SentenceTransformerEmbeddings(model_name="shibing624/text2vec-base-chinese")
    
    # 将文档列表转换为Document对象（确保doc是字符串类型）
    doc_objects = [Document(page_content=str(doc)) for doc in documents]
    
    # 使用Chroma创建向量索引
    index = Chroma.from_documents(doc_objects, embeddings)
    
    return index

# 使用示例：假设split_docs包含分片后的文档列表
index = build_index(split_docs)

# 打印索引信息（获取索引中文档数量）
print("Index built with", index._collection.count(), "document chunks.")

/var/folders/sd/5kzxkr7j1cq1356bqv4pt3840000gn/T/ipykernel_20219/59014988.py:10: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = SentenceTransformerEmbeddings(model_name="shibing624/text2vec-base-chinese")
/Users/queen/Downloads/llm_retrieval/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
INFO:sentence_transformers.SentenceTransformer:Use pytorch device_name: mps
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: shibing

Index built with 135 document chunks.


召回

In [4]:
def recall_documents(query, index, k=5):
    """
    A similarity retrieval is performed and the k most similar documents to the query are returned
    """
    return index.similarity_search(query, k=k)

# 使用示例
query = "2024年世界职业院校分为哪些赛道?"
retrieved_docs = recall_documents(query, index, k=5)

# 打印检索到的文档内容
for i, doc in enumerate(retrieved_docs):
    print(f"Document {i+1} content:", doc.page_content[:100])  # 打印每个文档的前100个字符


Document 1 content: page_content='2024 年世界职业院校技能大赛
制度汇编
世界职业院校技能大赛执行委员会（筹）
2024 年9 月
目
录
世界职业院校技能大赛管理规定与办法
组织机构与职能分工....
Document 2 content: page_content='类创意等。
3. 体现团队成员创新精神和创新能力。
10%
250
附件2
2024 年世界职业院校技能大赛
争夺赛国际组参赛视频要求
（建议稿）
（一）总体要求。参赛队伍
Document 3 content: page_content='督工作管理办法》执行。
189
十五、比赛观摩
比赛过程中应合理安排赛项现场观摩、线上直播，并明
确公开观摩的时间、形式以及应遵守的纪律要求等。
十六、比赛直播
阐述比赛除
Document 4 content: page_content='6
参赛管理办法
（试行）
为进一步发挥世界职业院校技能大赛（以下简称大赛）
培养学生综合技能的引领作用，促进提高职业教育人才培养
质量，充分展示广大职业学校学生的良好风貌
Document 5 content: page_content='（2）违反相关保密法规，泄露不应公开的信息。
6. 违反学术诚信
（1）存在抄袭、剽窃、盗用他人成果的行为。
（2）提供虚假或误导性的参赛材料。
7. 材料不完整
（1）未


重排

In [5]:
import os
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
from pathlib import Path

# 获取项目根目录（.env 所在目录）
root_dir = Path().resolve().parent

# 加载项目根目录下的 .env 文件中的 OPENROUTER_API_KEY
load_dotenv(dotenv_path=root_dir / ".env")

# 测试输出
api_key = os.getenv("OPENROUTER_API_KEY")
if api_key is None:
    raise ValueError("OPENROUTER_API_KEY not found in .env!")
print("✅ API key loaded:", api_key[:5] + "...")    # 👈 看看是不是 None

✅ API key loaded: sk-or...


In [6]:
from langchain_community.embeddings import ollama
from langchain_openai import ChatOpenAI
import os
from dotenv import load_dotenv
from pathlib import Path

# 加载项目根目录下的 .env 文件中的 OPENROUTER_API_KEY
root_dir = Path().resolve().parent
load_dotenv(dotenv_path=root_dir / ".env")
# 从 .env 读取 OPENROUTER_API_KEY，并设置成 OpenAI 兼容变量
os.environ["OPENAI_API_KEY"] = os.getenv("OPENROUTER_API_KEY")
os.environ["OPENAI_API_BASE"] = "https://openrouter.ai/api/v1"

# 获取 OPENROUTER_API_KEY 环境变量
api_key = os.getenv("OPENROUTER_API_KEY")

def rerank_documents(query, retrieved_docs):
    """
    Reorder retrieved documents using LLM or models
    """
    
    # 使用ChatOpenAI进行重排
    llm = ChatOpenAI(openai_api_key=api_key, model="openai/gpt-4o-mini")
    scores = []


    
    # 为每个文档生成相关性分数
    for doc in retrieved_docs:
        prompt = f"Query: {query}\nDocument: {doc.page_content}\n"
        
        # 构建适合ChatOpenAI的消息格式
        messages = [
            {"role": "system", "content": "You are an assistant helping to rank documents based on relevance."},
            {"role": "user", "content": prompt}
        ]
        
        # 获取相关性分数，确保传递正确的消息列表格式
        response = llm.invoke(messages)  # 确保使用 invoke 来调用
        
        # 提取模型生成的内容（直接访问 content 属性）
        score = response.content  # 获取生成的文本
        
        scores.append((score, doc))
    
    # 按照分数对文档进行排序
    sorted_docs = sorted(scores, key=lambda x: x[0], reverse=True)
    
    # 返回排序后的文档
    return [doc[1] for doc in sorted_docs]

# 使用示例：假设你已经调用了 recall_documents 获取了 retrieved_docs
query = "2024年世界职业院校分为哪些赛道?"
retrieved_docs = recall_documents(query, index, k=3)

# 重排检索到的文档
reranked_docs = rerank_documents(query, retrieved_docs)

# 打印重排后的文档内容
for i, doc in enumerate(reranked_docs):
    print(f"Reranked Document {i+1} content:", doc.page_content[:100])  # 打印每个文档的前100个字符


INFO:httpx:HTTP Request: POST https://openrouter.ai/api/v1/chat/completions "HTTP/1.1 402 Payment Required"


APIStatusError: Error code: 402 - {'error': {'message': 'This request requires more credits, or fewer max_tokens. You requested up to 16384 tokens, but can only afford 16163. To increase, visit https://openrouter.ai/settings/credits and upgrade to a paid account', 'code': 402, 'metadata': {'provider_name': None}}, 'user_id': 'user_2zM5yZHFZbyc3eeoXtSqnA2McVu'}

生成

In [39]:
from langchain_openai import ChatOpenAI
import os
from dotenv import load_dotenv
from pathlib import Path

# 加载项目根目录下的 .env 文件中的 OPENROUTER_API_KEY
root_dir = Path().resolve().parent
load_dotenv(dotenv_path=root_dir / ".env")

# 从 .env 读取 OPENROUTER_API_KEY，并设置成 OpenAI 兼容变量
os.environ["OPENAI_API_KEY"] = os.getenv("OPENROUTER_API_KEY")
os.environ["OPENAI_API_BASE"] = "https://openrouter.ai/api/v1"

# 获取 OPENROUTER_API_KEY 环境变量
api_key = os.getenv("OPENROUTER_API_KEY")

def generate_answer(query, top_docs):
    """
    基于查询和重排后的文档生成最终答案
    """
    # 将文档内容拼接为一个字符串（确保传递给模型的是一个长文本）
    documents_content = "\n".join([doc.page_content for doc in top_docs])
    
    # 格式化prompt，确保传递给模型的是一个字符串
    prompt = f"Answer the following question based on the retrieved documents:\n{documents_content}\n\nQuestion: {query}\nAnswer:"
    
    # 使用ChatOpenAI进行生成
    llm = ChatOpenAI(openai_api_key=api_key, model="openai/gpt-4o-mini")
    
    try:
        # 调用模型生成答案
        response = llm.invoke([{"role": "system", "content": "You are an assistant helping to answer questions based on documents."},
                               {"role": "user", "content": prompt}])
        
        # 获取生成的答案（通常是从 response.content 中提取）
        answer = response.content.strip()  # 提取生成的文本，去除多余的空格
        
        return answer
    except Exception as e:
        print(f"Error generating answer: {e}")
        return None


# 使用示例：假设你已经调用了 recall_documents 获取了 retrieved_docs 和 reranked_docs
query = "2024年世界职业院校分为哪些赛道?"
print("Generating answer for query:", query)

# 调用生成答案的函数
answer = generate_answer(query, reranked_docs)

# 打印生成的答案
if answer:
    print("Generated answer:", answer)
else:
    print("Failed to generate an answer.")


Generating answer for query: 2024年世界职业院校分为哪些赛道?


INFO:httpx:HTTP Request: POST https://openrouter.ai/api/v1/chat/completions "HTTP/1.1 200 OK"


Generated answer: 2024年世界职业院校技能大赛共设置42个赛道。这些赛道涉及不同的专业领域，具体的赛道信息可能会在相关的赛事通知或指南中详细列出。


集成 FastMCP 创建 MCP 服务器

In [38]:
import os
from dotenv import load_dotenv
from pathlib import Path
from fastmcp import FastMCP

# 加载项目根目录下的 .env 文件中的 OPENROUTER_API_KEY
root_dir = Path().resolve().parent
load_dotenv(dotenv_path=root_dir / ".env")

# 获取 OPENROUTER_API_KEY 环境变量
api_key = os.getenv("OPENROUTER_API_KEY")

# 定义知识库路径
knowledge_base_folder = "/Users/queen/Documents/VSCode/llm_retrieval/Local_knowledge_base"

# 创建 FastMCP 实例
mcp = FastMCP("localRetrieval", log_level="ERROR")

@mcp.tool(name="doc_init", description="A server for handling document retrieval and answer generation")
async def doc_init(self, knowledge_base_folder):
    """
    异步文档初始化，构建文档索引
    """
    self.index = build_index(knowledge_base_folder)  # 构建文档索引

@mcp.tool(name="handle_query", description="Handle a query by recalling, reranking documents and generating an answer")
async def handle_query(self, query):
    """
    处理查询，完成文档召回、重排与答案生成的过程
    """
    # Step 1: 召回文档
    retrieved_docs = await recall_documents(query, self.index)  # 异步召回文档
    
    # Step 2: 重排文档
    reranked_docs = await rerank_documents(query, retrieved_docs)  # 异步重排文档
    
    # Step 3: 生成答案
    answer = await generate_answer(query, reranked_docs)  # 异步生成答案
    
    return answer

# 确保 mcp.run() 运行时不与已有事件循环冲突
if __name__ == "__main__":
    import asyncio
    
    # 检查是否已有事件循环在运行
    if not asyncio.get_event_loop().is_running():
        # 如果没有事件循环，直接调用 run()
        mcp.run(transport='stdio')
    else:
        # 如果已经有事件循环运行，FastMCP 自动管理事件循环
        print("Event loop is already running. FastMCP will be managed automatically.")

Event loop is already running. FastMCP will be managed automatically.


/Users/queen/Documents/VSCode/llm_retrieval/.venv/lib/python3.13/site-packages/fastmcp/server/server.py:213: DeprecationWarning: Providing `log_level` when creating a server is deprecated. Provide it when calling `run` or as a global setting instead.
  self._handle_deprecated_settings(
